In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data=pd.read_csv("/kaggle/input/steam-store-games/steam.csv")
data=data.drop(["release_date", "steamspy_tags", "appid", "name", "english", "steamspy_tags"], axis=1)

In [3]:
platforms = data['platforms'].str.get_dummies(sep=';')
genres = data['genres'].str.get_dummies(sep=';')
categories = data['categories'].str.get_dummies(sep=';')


from sklearn.model_selection import train_test_split
y = data.owners
X = data.drop(['owners', 'categories', 'genres', 'platforms'], axis = 1)

X = pd.concat([X,platforms, genres, categories],axis=1)
# X.info()

In [4]:
def point_estimation(r):
    rng = r.split('-')
    rng[0] = int(rng[0])
    rng[1] = int(rng[1])
    return (rng[0] + rng[1]) / 2

lst = []
for i in range(len(y)):
    lst.append(point_estimation(y[i]))

y = pd.DataFrame(lst)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, 
                                                                random_state=0)


In [5]:
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error
from sklearn import linear_model
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression

model1 = RandomForestRegressor(n_estimators=100, random_state=0, verbose = 2)
model2 = LogisticRegression(solver='liblinear')
model3 = XGBRegressor()
model4 = lassoReg = linear_model.Lasso(alpha=0.1)
model5 = LinearRegression(verbose = 2)

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, object_cols)
    ])

my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model3)
                             ])

my_pipeline.fit(X_train, y_train[0])
preds = my_pipeline.predict(X_valid)
score = mean_absolute_error(y_valid[0], preds)
print('MAE:', score)

TypeError: __init__() got an unexpected keyword argument 'verbose'